In [1]:
from sympy import *
#import matplotlib.pyplot as plt
#import numpy as np
from IPython.display import display
init_printing(use_latex='mathjax')

def clean(expr):
    funcs = expr.atoms(Function)
    reps = {}
    for f in funcs:
        reps[f] = Symbol(f.name)
    return expr.subs(reps)

def clean_dict(d):
    return {k:clean(v) for k,v in d.items()}

# IMMD

This notebook is used to demonstrate how the method of multiples scales is used in an actual research environment.

This represents the bulk of the calculations in deriving the epynomous model of "Intraseaonsal Multi-Scale Moist Dynamics of the Tropical Troposhere," by Biello and Majda (2010) which can be found in section 3 and through Appendix A of the paper. This code exemplifies how such asymptotic methods can be automated.

In [2]:
#create variables (3.1ff)
epsilon = symbols('epsilon')
drag = {'u':symbols('d_u'),'theta':symbols('d_\\theta')}
x, X, y, t, T, z = symbols('x X y t T z')

#Create functions (3.3 and 3.7)
fields = ['u','v','w','p','\\theta','H']
big = {c: Function(f'{c.upper()}')(X,T,y,z) for c in fields}
mean = {c: Function(f'\\langle\\bar {c}\\rangle')(X,T,y,z) for c in fields}
#two ways of defining the perturbation terms
tilde = {c: Function(f'\\tilde{{ {c} }} ')(X,t,T,y,z) for c in fields}
pert = {c: Function(f'{c}_1')(x,X,t,T,y,z) for c in fields}
#tilde = {c: 0 for c in fields}
#pert = {c: Function(f'{c}_1^*')(x,X,t,T,y,z) for c in fields}
o2 = {c: Function(f'{c}_2')(x,X,t,T,y,z) for c in fields}

#Helper function to allow for these to be removed from expansions by operations
#used to emulate the filtering by (3.8-3.11)
s_filter = lambda d: {v:0 for v in d.values()}

#define the expansions (3.7)
u = big['u'] + epsilon*(tilde['u'] + pert['u'] + mean['u']) + epsilon**2*o2['u']
v = epsilon*(tilde['v'] + pert['v'] + mean['v']) + epsilon**2*o2['v']
w = epsilon*(tilde['w'] + pert['w'] + mean['w']) + epsilon**2*o2['w']
p = big['p'] + epsilon*(tilde['p'] + pert['p'] + mean['p']) + epsilon**2*o2['p']
H = (tilde['H'] + pert['H'] + mean['H']) + epsilon**2*o2['H']
theta = Function('Theta')(X,T,y,z) + \
        epsilon*(tilde['\\theta'] + pert['\\theta'] + mean['\\theta']) + epsilon**2*o2['\\theta']
#display(u,v,w,p,theta)

#define Derivatives (3.6, 3.15)
ddx = lambda f: f.diff(x) + epsilon*f.diff(X)
ddy = lambda f: f.diff(y)
ddz = lambda f: f.diff(z)
ddt = lambda f: f.diff(t) + epsilon*f.diff(T)
DDt = lambda f: ddt(f) + u*ddx(f) + v*ddy(f) + w*ddz(f)
cor = lambda f: y*f

#define the equations (3.2)
eqns = []
eqns.append(DDt(u) - cor(v) + ddx(p) + epsilon*drag['u']*u)
eqns.append(DDt(v) + cor(u) + ddy(p) + epsilon*drag['u']*v)
eqns.append(DDt(theta) + w - epsilon*H + epsilon*drag['theta']*theta)
eqns.append(ddz(p) - theta)
eqns.append(ddx(u) + ddy(v) + ddz(w))

eqns = [eqn.expand().collect(epsilon,evaluate=False) for eqn in eqns]

#Display O(1) eqns (3.12)
for eqn in eqns:
    if 1 in eqn.keys():
        display(clean(eqn[1]))


      d    
U⋅y + ──(P)
      dy   

     d    
-Θ + ──(P)
     dz   

In [3]:
#Compute O(epsilon) eqns, display mean equations (3.13)
mean_eqns = []
pert_eqns = []
tilde_eqns = []
for eqn in eqns:
    #filter terms simulate the averaging process
    mean_eqn = eqn[epsilon].subs(s_filter(pert)|s_filter(tilde)).simplify()
    pert_eqn = (eqn[epsilon] - mean_eqn).subs(s_filter(tilde)).simplify()
    tilde_eqn = (eqn[epsilon] - mean_eqn - pert_eqn).simplify()
    mean_eqns.append(mean_eqn)
    pert_eqns.append(pert_eqn)
    tilde_eqns.append(tilde_eqn)
    display(clean(mean_eqn))

         d                                                     d               ↪
U⋅dᵤ + U⋅──(U) - \langle\bar v\rangle⋅y + \langle\bar v\rangle⋅──(U) + \langle ↪
         dX                                                    dy              ↪

↪               d       d       d    
↪ \bar w\rangle⋅──(U) + ──(P) + ──(U)
↪               dz      dX      dT   

                         d                       
\langle\bar u\rangle⋅y + ──(\langle\bar p\rangle)
                         dy                      

               d                                                   d           ↪
Θ⋅d_\theta + U⋅──(Θ) - \langle\bar H\rangle + \langle\bar v\rangle⋅──(Θ) + \la ↪
               dX                                                  dy          ↪

↪                   d                              d    
↪ ngle\bar w\rangle⋅──(Θ) + \langle\bar w\rangle + ──(Θ)
↪                   dz                             dT   

                             d                       
-\langle\bar \theta\rangle + ──(\langle\bar p\rangle)
                             dz                      

d       d                          d                       
──(U) + ──(\langle\bar v\rangle) + ──(\langle\bar w\rangle)
dX      dy                         dz                      

We note that Equation (3.14) is not used in the above output, but could be done using a subs call (Exercise: implement this)

In [4]:
#Display perturbation O(epsilon) eqns (3.16)
for eqn in pert_eqns:
    display(clean(eqn))

  d                  d          d       d        d     
U⋅──(u₁) - v₁⋅y + v₁⋅──(U) + w₁⋅──(U) + ──(p₁) + ──(u₁)
  dx                 dy         dz      dx       dt    

  d               d        d     
U⋅──(v₁) + u₁⋅y + ──(p₁) + ──(v₁)
  dx              dy       dt    

        d                d          d            d          
-H₁ + U⋅──(\theta₁) + v₁⋅──(Θ) + w₁⋅──(Θ) + w₁ + ──(\theta₁)
        dx               dy         dz           dt         

           d     
-\theta₁ + ──(p₁)
           dz    

d        d        d     
──(u₁) + ──(v₁) + ──(w₁)
dx       dy       dz    

In [5]:
#Display tilde O(epsilon) eqns (3.18)
for eqn in tilde_eqns:
    display(clean(eqn))

                               d                    d       d               
-\tilde{ v } ⋅y + \tilde{ v } ⋅──(U) + \tilde{ w } ⋅──(U) + ──(\tilde{ u } )
                               dy                   dz      dt              

                 d                  d               
\tilde{ u } ⋅y + ──(\tilde{ p } ) + ──(\tilde{ v } )
                 dy                 dt              

                             d                    d                      d     ↪
-\tilde{ H }  + \tilde{ v } ⋅──(Θ) + \tilde{ w } ⋅──(Θ) + \tilde{ w }  + ──(\t ↪
                             dy                   dz                     dt    ↪

↪                 
↪ ilde{ \theta } )
↪                 

                     d               
-\tilde{ \theta }  + ──(\tilde{ p } )
                     dz              

d                  d               
──(\tilde{ v } ) + ──(\tilde{ w } )
dy                 dz              

At the $O(\epsilon^2)$ equations the raw output starts to appear a bit different from (3.19ff) Some of this can be attributed to different notations, the usage of incompressibility, but we first present the spatio-temporal average in (3.23) plus (3.22)

In [6]:
#Display O(epsilon^2) eqns
fluxes = [] 
for eqn in eqns:
    cond = eqn[epsilon**2].subs(s_filter(tilde)|s_filter(pert)).simplify()
    fluxes.append((eqn[epsilon**2]-cond).simplify())
    display(clean(cond))

  d                            d                                               ↪
U⋅──(\langle\bar u\rangle) + U⋅──(u₂) + \langle\bar u\rangle⋅dᵤ + \langle\bar  ↪
  dX                           dx                                              ↪

↪          d                            d                                      ↪
↪ u\rangle⋅──(U) + \langle\bar v\rangle⋅──(\langle\bar u\rangle) + \langle\bar ↪
↪          dX                           dy                                     ↪

↪           d                                    d          d       d          ↪
↪  w\rangle⋅──(\langle\bar u\rangle) - v₂⋅y + v₂⋅──(U) + w₂⋅──(U) + ──(\langle ↪
↪           dz                                   dy         dz      dX         ↪

↪                  d                          d        d     
↪ \bar p\rangle) + ──(\langle\bar u\rangle) + ──(p₂) + ──(u₂)
↪                  dT                         dx       dt    

  d                            d                                               ↪
U⋅──(\langle\bar v\rangle) + U⋅──(v₂) + \langle\bar v\rangle⋅dᵤ + \langle\bar  ↪
  dX                           dx                                              ↪

↪          d                                               d                   ↪
↪ v\rangle⋅──(\langle\bar v\rangle) + \langle\bar w\rangle⋅──(\langle\bar v\ra ↪
↪          dy                                              dz                  ↪

↪                d                          d        d     
↪ ngle) + u₂⋅y + ──(\langle\bar v\rangle) + ──(p₂) + ──(v₂)
↪                dT                         dy       dt    

  d                                 d                                          ↪
U⋅──(\langle\bar \theta\rangle) + U⋅──(\theta₂) + \langle\bar \theta\rangle⋅d_ ↪
  dX                                dx                                         ↪

↪                               d                            d                 ↪
↪ \theta + \langle\bar u\rangle⋅──(Θ) + \langle\bar v\rangle⋅──(\langle\bar \t ↪
↪                               dX                           dy                ↪

↪                                     d                                  d     ↪
↪ heta\rangle) + \langle\bar w\rangle⋅──(\langle\bar \theta\rangle) + v₂⋅──(Θ) ↪
↪                                     dz                                 dy    ↪

↪       d            d                               d          
↪  + w₂⋅──(Θ) + w₂ + ──(\langle\bar \theta\rangle) + ──(\theta₂)
↪       dz           dT                              dt         

           d     
-\theta₂ + ──(p₂)
           dz    

d                          d        d        d     
──(\langle\bar u\rangle) + ──(u₂) + ──(v₂) + ──(w₂)
dX                         dx       dy       dz    

Followed by all the remaining nonlinear terms. Most of these would be negated by the averaging process leaving only the fluxes, but unfortunately this must be done by hand. Even so, then end results of the hand-computed fluxes could readily be copy-pasted from the notebook into some $\LaTeX$ or equivalent document with all negated terms simply deleted, which is achieved in the commented line.

In [13]:
for flux in fluxes:
    display(clean(flux))
    #print(latex(clean(flux)))

  d                    d                             d                         ↪
U⋅──(\tilde{ u } ) + U⋅──(u₁) + \langle\bar u\rangle⋅──(u₁) + \langle\bar v\ra ↪
  dX                   dX                            dx                        ↪

↪      d                                       d                             d ↪
↪ ngle⋅──(\tilde{ u } ) + \langle\bar v\rangle⋅──(u₁) + \langle\bar w\rangle⋅─ ↪
↪      dy                                      dy                            d ↪

↪                                        d                                     ↪
↪ ─(\tilde{ u } ) + \langle\bar w\rangle⋅──(u₁) + \tilde{ u } ⋅dᵤ + \tilde{ u  ↪
↪ z                                      dz                                    ↪

↪    d                    d                     d                              ↪
↪ } ⋅──(U) + \tilde{ u } ⋅──(u₁) + \tilde{ v } ⋅──(\langle\bar u\rangle) + \ti ↪
↪    dX                   dx                    dy                             ↪

↪           d           

  d                    d                             d                         ↪
U⋅──(\tilde{ v } ) + U⋅──(v₁) + \langle\bar u\rangle⋅──(v₁) + \langle\bar v\ra ↪
  dX                   dX                            dx                        ↪

↪      d                                       d                             d ↪
↪ ngle⋅──(\tilde{ v } ) + \langle\bar v\rangle⋅──(v₁) + \langle\bar w\rangle⋅─ ↪
↪      dy                                      dy                            d ↪

↪                                        d                     d               ↪
↪ ─(\tilde{ v } ) + \langle\bar w\rangle⋅──(v₁) + \tilde{ u } ⋅──(v₁) + \tilde ↪
↪ z                                      dz                    dx              ↪

↪                          d                                       d           ↪
↪ { v } ⋅dᵤ + \tilde{ v } ⋅──(\langle\bar v\rangle) + \tilde{ v } ⋅──(\tilde{  ↪
↪                          dy                                      dy          ↪

↪                      d

  d               d                                            d               ↪
U⋅──(\theta₁) + U⋅──(\tilde{ \theta } ) + \langle\bar u\rangle⋅──(\theta₁) + \ ↪
  dX              dX                                           dx              ↪

↪                     d                                  d                     ↪
↪ langle\bar v\rangle⋅──(\theta₁) + \langle\bar v\rangle⋅──(\tilde{ \theta } ) ↪
↪                     dy                                 dy                    ↪

↪                         d                                  d                 ↪
↪  + \langle\bar w\rangle⋅──(\theta₁) + \langle\bar w\rangle⋅──(\tilde{ \theta ↪
↪                         dz                                 dz                ↪

↪                                                                     d        ↪
↪  } ) + \theta₁⋅d_\theta + \tilde{ \theta } ⋅d_\theta + \tilde{ u } ⋅──(Θ) +  ↪
↪                                                                     dX       ↪

↪              d        

0

d                  d     
──(\tilde{ u } ) + ──(u₁)
dX                 dX    